In [3]:
import requests
import pandas as pd


In [4]:
#Create a Python program to GET (consume) data from the above API endpoint for the loan application dataset.
got = requests.get('https://raw.githubusercontent.com/platformps/LoanDataset/main/loan_data.json')
df = pd.read_json(got.content)



In [ ]:
#Find the status code of the above API endpoint.
print(got.status_code)